In [13]:
from Learning import *
import torch
from transformers import GPT2ForSequenceClassification, ReformerModelWithLMHead
from pytorch_transformers import GPT2Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

I0121 05:44:40.287773 46184 modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0121 05:44:40.294755 46184 modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
# Put a custom module mimicking the LM head on top of the model and use the membership loss of the top-k words to fine-tune
# the full set of existing parameters, as should be the default in torch, and should be possible for ELMo 
# Top-k and the temperature of the final word prediction can also be parameters on the web interface ("randomness"?)

In [20]:
# Attach the LM head
model = GPT2LMHeadModel.from_pretrained('gpt2-large',
    output_hidden_states=True, output_attentions=True, cache_dir="C:\\Users\\alfew\\Code\\datas\\torch\\GPT2LMHead")

I0121 07:39:26.218024 46184 file_utils.py:238] https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-large-config.json not found in cache or force_download set to True, downloading to C:\Users\alfew\AppData\Local\Temp\tmp0j9iqn9u

100%|█████████████████████████████████████████████████████████████████████████████| 764/764 [00:00<00:00, 383169.71B/s]I0121 07:39:26.617969 46184 file_utils.py:251] copying C:\Users\alfew\AppData\Local\Temp\tmp0j9iqn9u to cache at C:\Users\alfew\Code\datas\torch\GPT2LMHead\c8f887cdfff4327916f4b7ed06a379c0add42bd9c66e1fe3b4a5a8525a4b2678.7a56eb872b7d0abfad5ae7e76f318ac26b189fad23442b4017703dd0b946115a
I0121 07:39:26.619965 46184 file_utils.py:255] creating metadata file for C:\Users\alfew\Code\datas\torch\GPT2LMHead\c8f887cdfff4327916f4b7ed06a379c0add42bd9c66e1fe3b4a5a8525a4b2678.7a56eb872b7d0abfad5ae7e76f318ac26b189fad23442b4017703dd0b946115a
I0121 07:39:26.621958 46184 file_utils.py:264] removing temp file C:\Users\alfew\AppData\Local\Temp\tmp0j9iqn9u
I0

In [68]:
# Get model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text1 = "OMG What<|endoftext|><|endoftext|><|endoftext|>"
text2 = "OMG What is the<|endoftext|>"
text3 = "OMG What is the thing"
tokens1 = tokenizer.encode(text1)
tokens2 = tokenizer.encode(text2)
tokens3 = tokenizer.encode(text3)
tokens_tensor = torch.tensor([tokens1, tokens2, tokens3])
model.eval() # Dropout = 0
tokens_tensor = tokens_tensor.to(device)
model.to(device)
outputs = None
with torch.no_grad():
    outputs = model(tokens_tensor)
# top_next = [self.tokenizer.decode(i.item()).strip() for i in probs.topk(k)[1]]

I0121 07:59:25.226185 46184 tokenization_utils.py:384] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at C:\Users\alfew\.cache\torch\pytorch_transformers\f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0121 07:59:25.229176 46184 tokenization_utils.py:384] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at C:\Users\alfew\.cache\torch\pytorch_transformers\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [69]:
len(outputs), outputs[0].shape, len(outputs[1]), outputs[1][-1].shape, \
    len(outputs[2]), outputs[2][-1].shape, len(outputs[3]), outputs[3][-1].shape, type(outputs)

(4,
 torch.Size([3, 5, 50257]),
 36,
 torch.Size([2, 3, 20, 5, 64]),
 37,
 torch.Size([3, 5, 1280]),
 36,
 torch.Size([3, 20, 5, 5]),
 tuple)

In [70]:
model.config.hidden_size

1280

In [54]:
np.asarray([torch.Tensor.cpu(outputs[2][i]).detach().numpy() for i in range(36)]).shape

(36, 3, 4, 1280)

In [55]:
np.mean(np.asarray([torch.Tensor.cpu(outputs[2][i]).detach().numpy() for i in range(36)]), axis=0).shape

(3, 4, 1280)

In [67]:
torch.Tensor.cpu(outputs[2][-1]).detach().numpy()

array([[[-0.01064283,  0.44653678,  0.11003283, ...,  1.1178464 ,
          2.090374  ,  0.3782196 ],
        [ 0.20237704,  0.24751762,  0.25801387, ..., -0.28999507,
         -0.48244268, -1.157095  ],
        [ 0.5048916 ,  0.31524453,  0.3927654 , ..., -0.22111693,
         -0.34072432, -0.9919133 ],
        [ 0.6727123 ,  0.3622866 ,  0.42447126, ..., -0.21644257,
         -0.34485823, -0.9407382 ]],

       [[-0.01064283,  0.44653678,  0.11003283, ...,  1.1178464 ,
          2.090374  ,  0.3782196 ],
        [ 0.97022164, -0.6258169 , -1.1055696 , ..., -3.196596  ,
         -2.3969207 , -0.35349554],
        [-0.2373758 , -1.3782818 , -0.46534598, ..., -3.116994  ,
         -1.604099  , -2.5942338 ],
        [ 0.35378096,  0.29732588,  0.265001  , ..., -0.33094501,
         -0.4192703 , -1.7629004 ]],

       [[-0.01064283,  0.44653678,  0.11003283, ...,  1.1178464 ,
          2.090374  ,  0.3782196 ],
        [ 0.97022164, -0.6258169 , -1.1055696 , ..., -3.196596  ,
         -2.

In [52]:
outputs[2][:36]

(tensor([[[-0.0008, -0.0017, -0.0446,  ..., -0.0291,  0.0799,  0.0218],
          [-0.0464,  0.0878, -0.0578,  ..., -0.0038, -0.0319, -0.0305],
          [-0.0427,  0.0879, -0.0531,  ..., -0.0025, -0.0463, -0.0242],
          [-0.0432,  0.0779, -0.0525,  ..., -0.0012, -0.0490, -0.0367]],
 
         [[-0.0008, -0.0017, -0.0446,  ..., -0.0291,  0.0799,  0.0218],
          [-0.0362, -0.0014, -0.0363,  ...,  0.0303,  0.0433, -0.0305],
          [ 0.0145, -0.0154, -0.0251,  ...,  0.0438, -0.0272, -0.0268],
          [-0.0432,  0.0779, -0.0525,  ..., -0.0012, -0.0490, -0.0367]],
 
         [[-0.0008, -0.0017, -0.0446,  ..., -0.0291,  0.0799,  0.0218],
          [-0.0362, -0.0014, -0.0363,  ...,  0.0303,  0.0433, -0.0305],
          [ 0.0145, -0.0154, -0.0251,  ...,  0.0438, -0.0272, -0.0268],
          [-0.0582, -0.0182,  0.0602,  ..., -0.0272, -0.0019,  0.0445]]]),
 tensor([[[ 0.0082,  0.0319,  0.0197,  ..., -0.3442, -0.7590,  0.2280],
          [ 0.0160,  0.8350, -0.1914,  ...,  0.7370, -0

In [ ]:
model = torch.hub.load('huggingface/pytorch-transformers', 'modelWithLMHead', 'bert-base-uncased', output_attentions=True)  # Update configuration during loading

In [50]:
# tokenizer = AutoTokenizer.from_pretrained("google/reformer-enwik8")
model = ReformerModelWithLMHead.from_pretrained("google/reformer-enwik8")
text = "What is the"
# tokens = tokenizer.encode(text)
tokens_tensor = torch.tensor(text)
model.eval() # Dropout = 0
tokens_tensor = tokens_tensor.to(device)
model.to(device)
outputs = None
with torch.no_grad():
    outputs = model(tokens_tensor)
# top_next = [self.tokenizer.decode(i.item()).strip() for i in probs.topk(k)[1]]

I0120 20:18:48.096265 46352 filelock.py:274] Lock 2959410044488 acquired on C:\Users\alfew/.cache\huggingface\transformers\c212088c2fedccd67006754bd70fd4813266e5ad9f0231484a709c39ba7f20d2.a162535b412dee5382cee738fc4b5cc6a46cf215ca642b18ce2a94899f297e4e.lock


I0120 20:18:48.568516 46352 filelock.py:318] Lock 2959410044488 released on C:\Users\alfew/.cache\huggingface\transformers\c212088c2fedccd67006754bd70fd4813266e5ad9f0231484a709c39ba7f20d2.a162535b412dee5382cee738fc4b5cc6a46cf215ca642b18ce2a94899f297e4e.lock
I0120 20:18:48.983909 46352 filelock.py:274] Lock 2959410000728 acquired on C:\Users\alfew/.cache\huggingface\transformers\7ff5ad1e314f1027cfefc8c6e06dcd7378d12a896e8f8de78766394b7886a7de.07a38acfd69ce3ffe1428b5a9020b8da55d4a98b612c19d7c01401630b057df4.lock


I0120 20:21:24.091155 46352 filelock.py:318] Lock 2959410000728 released on C:\Users\alfew/.cache\huggingface\transformers\7ff5ad1e314f1027cfefc8c6e06dcd7378d12a896e8f8de78766394b7886a7de.07a38acfd69ce3ffe1428b5a9020b8da55d4a98b612c19d7c01401630b057df4.lock
c:\python36\lib\site-packages\torch\nn\modules\container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


TypeError: new(): invalid data type 'str'

In [43]:
outputs[0].shape, len(outputs[1]), outputs[1][0].shape

(torch.Size([1, 3, 50257]), 12, torch.Size([2, 1, 12, 3, 64]))

In [35]:
library.autocorrect_word("lowenergy"), library.autocorrect_word("london"), library.autocorrect_word("gOrdon")

NameError: name 'library' is not defined

In [5]:
ngrams_db = load_ngram_counts()


Loading ngram counts database... 1120379 entries
Loading ngram counts database took 3.025479555130005 seconds.


In [6]:
library.import_autocorrect_1grams(ngrams_db)

Importing 1gram capitalisation autocorrection index...
Completed 1grams import.


In [8]:
library.autocorrect_word("lowenergy"), library.autocorrect_word("london"), library.autocorrect_word("wifi")

('low-energy', 'London', 'WiFi')

In [10]:
library.get_wrepi("low-energy")

(115072,
 [-0.045132,
  0.40205,
  -0.094554,
  0.5083300000000001,
  -0.37155,
  0.25371,
  -0.14272,
  -0.022713,
  -0.062012,
  -0.74692,
  0.31874,
  -0.013853,
  -0.23779,
  0.28037,
  -0.1967,
  0.19097999999999998,
  0.74578,
  0.76653,
  0.38413,
  -0.93311,
  0.47248,
  -0.0098441,
  -0.24595,
  -0.12468,
  -0.8135899999999999,
  0.22905,
  0.15031,
  -0.29249,
  -0.17925,
  0.87231,
  0.10582000000000001,
  -0.76927,
  -0.33906,
  0.17722000000000002,
  0.36939,
  0.45053000000000004,
  -0.5226,
  -0.49111000000000005,
  0.22758,
  0.15678,
  -0.41483000000000003,
  0.19166,
  -0.25333,
  0.0014856,
  -0.56901,
  0.34595,
  -0.26116,
  -0.52915,
  -0.53135,
  0.096401,
  -0.21930999999999998,
  0.38724000000000003,
  -0.46863,
  0.3214,
  -0.047859,
  0.0271,
  0.038146,
  -0.07319500000000001,
  0.37725,
  0.6048,
  0.49513999999999997,
  -0.10355999999999999,
  0.1035,
  -0.12419000000000001,
  0.11182,
  -0.142,
  -0.15406,
  -0.32849,
  -0.1193,
  0.11332,
  0.39384,
  -0

In [6]:
fruits = ["Gaussian process", "decision tree", "neural network", "perceptron", "random forest", "linear regression", "mixture model", "Gaussian mixture model", "matrix factorization", "Restricted Boltzmann Machine", "deep learning", "Support Vector Machine", "Long Short-Term Memory", "convolutional neural network", "recurrent neural network", "transformer", "attention", "multi-head attention", "residual connections", "dropout", "batch normalisation", "GAN", "generative adversarial network", "LDA", "latent Dirichlet allocation", "HMM", "hidden Markov model", "GPT", "GloVe", "word2vec", "BERT", "ELMo", "DeepDream", "variational autoencoder", "logistic regression", "fastText", "k nearest neighbours", "k means", "AdaBoost", "belief network", "graphical model", "genetic algorithm", "autoencoder", "inception", "monte carlo", "gradient descent", "spiking neural network", "image segmentation", "U-Net", "ReLU", "softmax", "Expectation–maximization", "density estimation", "inverse entailment", "inductive logic programming", "computer vision", "reinforcement learning", "recommender system", "optical character recognition", "facial recognition", "emotion recognition", "upsampling", "super resolution", "inverse reinforcement learning", "temporal difference learning", "optimal control"]

In [16]:
np.random.shuffle(fruits)
print(len(fruits))
"A list of ML algorithms: " + ', '.join(fruits) + ', '

66


'A list of ML algorithms: Gaussian mixture model, logistic regression, ReLU, LDA, emotion recognition, image segmentation, gradient descent, DeepDream, AdaBoost, generative adversarial network, GloVe, convolutional neural network, decision tree, transformer, neural network, GPT, U-Net, facial recognition, Long Short-Term Memory, recurrent neural network, inverse entailment, Expectation–maximization, optimal control, word2vec, density estimation, linear regression, latent Dirichlet allocation, k nearest neighbours, matrix factorization, genetic algorithm, batch normalisation, GAN, BERT, inductive logic programming, perceptron, belief network, optical character recognition, monte carlo, computer vision, inverse reinforcement learning, k means, Gaussian process, fastText, deep learning, super resolution, dropout, HMM, Restricted Boltzmann Machine, softmax, attention, autoencoder, multi-head attention, upsampling, recommender system, inception, ELMo, variational autoencoder, residual conne

In [4]:
fruits=["apple","orange","grapefruit","peach","lemon","lime","pineapple","kiwi","mango","papaya","watermelon",
        "strawberry","pomegranate","melon","plum","pear","pumpkin","apricot","cherry"]
np.random.shuffle(fruits)
print(len(fruits))
"List of round fruits: " + ', '.join(fruits) + ', '

19


'List of round fruits: strawberry, lemon, cherry, grapefruit, kiwi, pomegranate, pumpkin, peach, orange, apricot, plum, watermelon, mango, papaya, pear, pineapple, melon, lime, apple, '

In [24]:
fruits=["fox", "dolphin", "elephant", "dog", "wolf", "monkey", "lizard", "rabbit", "rat", "monkey", "cat",
        "bird", "frog", "horse", "fish", "whale", "shark", "turtle", "spider", "bat",  "butterfly", "moth", "beetle", 
        "worm", "snake", "fly"]
np.random.shuffle(fruits)
print(len(fruits))
"Long list of well-known wild animals: " + ', '.join(fruits) + ', '

26


'Long list of well-known wild animals: lizard, beetle, dolphin, whale, wolf, shark, fox, rat, moth, spider, rabbit, monkey, butterfly, bird, snake, fly, bat, monkey, turtle, frog, elephant, horse, dog, fish, worm, cat, '

In [72]:
ngrams_db_precomp = pd.read_csv(ngrams_csv_precomp, index_col="root",
    keep_default_na=False, usecols=["root", "word", "type", "total"], dtype={
    **{k: str for k in ["word", "root", "type"]},
    **{k: float for k in ["total", "total_b", "last5", "last10", "last20"]},
    **{k: float for k in ["mean", "mode", "min", "max"]}})
ngrams_db_precomp = ngrams_db_precomp.sort_index()

In [76]:
ngc_cols = ['type', 'ngc', 'nbc', 'mean', 'mean_mode', 'mode_mode',
            'min', 'max', 'last5', 'last10', 'last20']
ngrams_db = pd.read_csv(data_dir + "ngram_counts.csv", index_col="word", encoding='utf-8',
        keep_default_na=False, dtype={k: str for k in ngc_cols})

In [71]:
ngrams_db_precomp

,type,total
word,,
007,NUM,56.0
007,.,639.0
007,X,47758.0
007,VERB,1172.0
007,ADV,109.0
007,NOUN,193044.0
007,NUM,1340920.0
007,DET,116.0
007,PRON,108.0


In [ ]:
ngrams_pretotals = ngrams_db_precomp#[["type", "total"]]
n_doing = len(ngrams_db.index)
i = 0
start_time = time.time()
new_words, new_types = [], []
for w in list(ngrams_db.index):
    i += 1
    w = w.lower().replace('-', '')
    x = ngrams_pretotals.loc[w]
    if isinstance(x, pd.Series):
        x = pd.DataFrame(x).T
        x.index.name = "root"
        
    x__ = []
    x_v = {}  # Word part of speech variants stored for top type extraction
    uniq_variants = list(x.word.unique())
    for w_ in uniq_variants:
        x_ = x[x.word == w_]
        x_v[w_] = x_
        x__.append(x_.iloc[np.argmax(x_["total"].values)])
    x = pd.DataFrame(x__)
    x_ = x[x.type == '?']
    if x_.shape[0] < 1: x_ = x
    arg_max = x_.iloc[np.argmax(x_["total"].values)]
    word = arg_max.word

    pos_filt = x_v[word].query("type == @pos_tags")
    pos_max = pos_filt.iloc[np.argmax(pos_filt["total"].values)].type \
        if pos_filt.shape[0] > 0 else '?'
    
    new_words.append(word)
    new_types.append(pos_max)
    
    if (i % 1000 == 0 or i == (n_doing - 1)) and i > 0:
        pc = round(100.0 * (float(i) / float(n_doing)), 2)
        fr = float(n_doing - i) / float(i)
        tr = (time.time() - start_time) * fr
        tr = (str(round(tr / 60, 2)) + ' minute' if tr > 120 \
              else str(round(tr, 2)) + ' second') + "s remaining)      "
        sys_print("\rFixed word ngram types: " + str(i) + " / " + \
            str(n_doing) + " (" + str(pc) + "%, " + tr)
ngrams_db.index = new_words
ngrams_db.index.name = "word"
ngrams_db.loc[:, "type"] = new_types

Fixed word ngram types: 226000 / 1120388 (20.17%, 66.49 minutes remaining)      

In [56]:
for k in ngrams_db.index:
    if k is None or not isinstance(k, str) or k == '':
        print(k, type(k))

In [61]:
from copy import deepcopy
ngrams_db_exp = deepcopy(ngrams_db)

In [63]:
ngrams_db_exp.index = ["word" + str(i) for i in range(len(ngrams_db_exp.index))]

In [68]:
ngrams_db_exp.index.name = "word"
ngrams_db_exp.index


Index(['word0', 'word1', 'word2', 'word3', 'word4', 'word5', 'word6', 'word7',
       'word8', 'word9',
       ...
       'word1120426', 'word1120427', 'word1120428', 'word1120429',
       'word1120430', 'word1120431', 'word1120432', 'word1120433',
       'word1120434', 'word1120435'],
      dtype='object', name='word', length=1120436)

In [69]:
ngrams_db.index

Index(['007', 'a', 'a0', 'a00', 'a000', 'a0001', 'a001', 'a001438', 'a002',
       'a003',
       ...
       'wallins', 'wannier', 'webvision', 'wizened', 'womej', 'yingfa',
       'zemplar', 'ziffdavis', 'zth', 'front'],
      dtype='object', name='word', length=1120436)

In [49]:
ngrams_db_preall = pd.read_csv(ngrams_csv_precomp, index_col="word",
    keep_default_na=False, dtype={
    **{k: str for k in ["word", "root", "type"]},
    **{k: float for k in ["total", "total_b", "last5", "last10", "last20"]},
    **{k: float for k in ["mean", "mode", "min", "max"]}})
ngrams_db_preall = ngrams_db_preall.sort_index()

In [53]:
ngrams_db_preall.loc[""]

KeyError: 'the label [] is not in the [index]'

In [54]:
ngrams_db_precomp

,type,total
word,,
007,NUM,56.0
007,.,639.0
007,X,47758.0
007,VERB,1172.0
007,ADV,109.0
007,NOUN,193044.0
007,NUM,1340920.0
007,DET,116.0
007,PRON,108.0


In [36]:
ngrams_db.loc["5221"]

type         NaN
ngc          NaN
nbc          NaN
mean         NaN
mean_mode    NaN
mode_mode    NaN
min          NaN
max          NaN
last5        NaN
last10       NaN
last20       NaN
Name: 5221, dtype: object

In [39]:
%timeit ngrams_db.loc["front", "type"] = "?"

385 ms ± 9.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit ngrams_db_precomp.loc["a001438"]

116 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
list(ngrams_db_precomp.loc[["a001438"]].index.unique())

['a001438']

In [18]:
a = pd.DataFrame(ngrams_db_precomp.loc["a001438"]).T
a.index.name = "word"
a

,type,total
word,,
a001438,?,56


In [24]:
list(a.index.unique())

['a001438']

In [27]:
type(ngrams_db_precomp.loc["a001438"]), type(ngrams_db_precomp.loc["a00"])

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [31]:
isinstance(ngrams_db_precomp.loc["a001438"], pd.Series)

True

In [30]:
pd.Series == pd.core.series.Series

True

In [62]:
qmarks = ['???' + str(i) for i in range(len(ngrams_db))]

In [33]:
Out[19]

type                 ?
ngc          218510629
nbc           14518463
mean          1969.817
mean_mode     2013.696
mode_mode         2014
min               1472
max               2019
last5         23733049
last10        50738847
last20        84696606
Name: front, dtype: object

In [29]:
a = ngrams_db_precomp[ngrams_db_precomp["word"] == "front"][["type", "total"]].query("type == @pos_tags")
a.iloc[np.argmax(a["total"].values)].type

'NOUN'

In [27]:
ngrams_db_precomp.loc["front"], ngrams_db_precomp.loc["front"].iloc[np.argmax(ngrams_db_precomp.loc["front"].total.values)]

(        word      type        total     total_b      mean    mode     min  \
 root                                                                        
 front  front       ADJ         87.0        47.0  2008.149  2011.0  1929.0   
 front  front       ADJ        265.0       254.0  1928.015  1919.0  1821.0   
 front  Front      NOUN        131.0       122.0  1955.550  2011.0  1860.0   
 front  fRont      NOUN        115.0        90.0  2000.452  2014.0  1851.0   
 front  front      NOUN         74.0        65.0  1987.757  2006.0  1856.0   
 front  front  iterator        380.0        93.0  2005.411  1999.0  1995.0   
 front  FRONT      NOUN    1494728.0    508664.0  1963.749  1985.0  1500.0   
 front  front         ?  198987348.0  11041128.0  1969.553  2014.0  1475.0   
 front  FRONT         ?    1650539.0    544028.0  1964.323  1985.0  1500.0   
 front  FroNT       ADJ        140.0        43.0  1959.779  1944.0  1805.0   
 front  front     wheel        165.0        44.0  2009.594  2015

In [50]:
ngrams_db[["type", "total"]].groupby("type").sum().sum()

total    3119245190725
dtype: int64

In [48]:
ngrams_db[["type", "total"]].groupby("type").sum().sort_values("total")[-55:]

,total
type,
A,46441
not,46499
0,46500
time,47759
nd,48566
3,49227
HOME,49238
I,49533
value,50423


In [52]:
ngrams_db[["type", "total"]].groupby("type").sum().sort_values("total")[-15:] / 31192451907.25

,total
type,
A,0.000001
not,0.000001
0,0.000001
time,0.000002
nd,0.000002
3,0.000002
HOME,0.000002
I,0.000002
value,0.000002


In [75]:
sorted(['c', 'a', 'd'])

['a', 'c', 'd']

In [76]:
[w for w in ngrams_db.root.unique() if type(w) == float]

[nan]

In [68]:
pos_tags = [
    "X",       #   4562969893          (   0.15%   )
    "NUM",     # 10359909952           (   0.33%   )
    "PRT",     # 45361608405           (   1.45%   )
    "CONJ",    #    60763767404        (   1.95%   )
    "ADV",     # 71401527469           (   2.29%   )
    "PRON",    #    75534024221        (   2.42%   )
    "ADJ",     # 128073334803          (   4.11%   )
    "DET",     # 184719428708          (   5.92%   )
    "ADP",     # 216548456584          (   6.94%   )
    "VERB",    #    249754507701       (   8.00%   )
    "NOUN",    #   1559809008954       (   16.42%  )
]

In [72]:
ngrams_db[["type", "total"]].groupby("type").sum().sort_values("total")[-15:].query("type == @pos_tags").shape[0]

0

In [58]:
ngrams_db.query("type == @pos_tags")

UndefinedVariableError: local variable 'pos_tag' is not defined

In [252]:
library = pd.read_table(data_dir + word_rep_dir + "glove.840B/glove.840B.300d.txt", header=None, sep=' ',
                encoding="utf8", index_col=0, quoting=csv.QUOTE_NONE)

In [253]:
library.shape

(2196017, 300)

In [233]:
library.shape[0]

1193514

In [246]:
for x in library.index.dropna().values:
#     print(type(x))
    if type(x) != str:
        print(x, type(x))

In [254]:
print(library.shape[0])
library = library.loc[library.index.dropna()]
print(library.shape)

2196017
(2196011, 300)


In [1]:
library.iloc[0]

NameError: name 'library' is not defined

In [242]:
library.loc[float('nan')]

KeyError: nan

In [240]:
np.nan

nan

In [236]:
library.loc[np.nan].delete()

KeyError: nan

In [123]:
100 * ngrams_db.root.unique().shape[0] / ngrams_db.shape[0], ngrams_db.shape[0]

(8.841758803619257, 4338017)

In [176]:
ngrams_db.iloc[234], ngrams_db.loc[ngrams_db.index[234]],

(root         chofu
 total         1093
 total_b        625
 mean       1976.07
 mode          1994
 min           1885
 max           2019
 last5            6
 last10          29
 last20         156
 Name: CHOFU, dtype: object,
         root  total  total_b      mean    mode   min   max  last5  last10  \
 word                                                                        
 CHOFU  chofu   1093      625  1976.067  1994.0  1885  2019      6      29   
 CHOFU  chofu   1092      624  1976.137  1994.0  1885  2019      6      29   
 
        last20  
 word           
 CHOFU     156  
 CHOFU     156  )

In [166]:
a = pd.DataFrame([ngrams_db.iloc[234], ngrams_db.iloc[2343]])
a.total * a["mean"]

CHOFU     2159841.231
BuCHAN     337951.941
dtype: float64

In [172]:
list(ngrams_db.iloc[234])

['chofu', 1093, 625, 1976.067, 1994.0, 1885, 2019, 6, 29, 156]

In [171]:
for w in list(ngrams_db.root.unique()):
    x = ngrams_db[ngrams_db['root'] == w]
    total = sum(x['total'])
    print(x[["root", "total"]].sort_values("word"))
    x__ = []
    for w_ in list(x.index.unique()):
        x_ = x[x.index == w_]
        i = np.argmax(list(x_["total"].values))
        x__.append(x_.iloc[i])
    x = pd.DataFrame(x__)
    total = sum(x["total"])
    total_b = sum(x["total_b"])
    mean = sum(x["mean"] * (x["total"] / total))
    mean_mode = sum(x["mode"] * (x["total"] / total))
    mode_mode = x.iloc[np.argmax(x["total"].values)]["mode"]
    print(total, total_b, mean, mean_mode, mode_mode)
#         print(x_)
#         print(i)
#         print(x_.iloc[i].values)
#         print(x_.iloc[i]["total"])
    break

        root   total
word                
BEHAR  behar   17982
BEHAR  behar   17960
BeHar  behar     235
BeHar  behar     235
Behar  behar  622098
Behar  behar      94
Behar  behar  621861
Behar  behar      54
Behar  behar      47
behar  behar     290
behar  behar    1608
behar  behar    2628
behar  behar     110
behar  behar    2414
behar  behar    7196
behar  behar     141
647511 174312 1946.9633272577607 2012.5242891626551 2013.0


In [136]:
622098 - (621861 + 94 + 54 + 47)

42

In [157]:
a = set([w for w in ngrams_db.index])
len(a)

1448081

In [158]:
ngrams_db.shape[0]

4338017

In [72]:
word_rep_data = data_dir + word_rep_dir + "glove.twitter.27B/glove.twitter.27B.25d.txt"
library = WordRepLibrary(word_rep_data)

Loading word representation library 'glove.twitter.27B.25d.txt'
Loading word representation library took 5.1542768478393555 seconds.



In [74]:
library.get_exists("concepteur"), library.get_exists("duck")

(True, True)

In [87]:
def strind(s, c):
    try:
        return s.index(c)
    except:
        return

In [90]:
"sfd_asd"[:strind("sfd_asd", '_')]

'sfd'

In [97]:
int(1223452345233452345234523451234123434534563456345645362456245643345452343.534)

1223452345233452259389135371294300936956599211557729001369731293811572736

In [ ]:
import enchant
d = enchant.Dict("en_GB")

In [62]:
d.check("concepteur")

False

In [69]:
from nltk.corpus import words
g = set(words.words())

In [66]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\alfew\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [71]:
"dispatient" in g

False

In [21]:
string.ascii_lowercase + string.ascii_uppercase + '0123456789' + '-'

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [24]:
"sdf_".strip()

'sdf_'

In [8]:
df = pd.read_csv(data_csv, sep='\t', index_col=0)

In [18]:
pd.Timestamp.now().timestamp()

1608134459.587759

In [2]:
google_search_count("blah")

AttributeError: 'NoneType' object has no attribute 'text'

In [3]:
from googlesearch import search

In [4]:
search("blah",        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = None,  # Last result to retrieve
                pause = 2.0,  # Lapse between HTTP requests
               )

<generator object search at 0x000001B28A162EB8>

In [7]:
my_results_list = []
for i in search("blah",        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = None,  # Last result to retrieve
                pause = 2.0,  # Lapse between HTTP requests
               ):
    my_results_list.append(i)
    print(i)

https://www.merriam-webster.com/dictionary/blah
https://www.urbandictionary.com/define.php?term=blah
https://www.dictionary.com/browse/blah
https://www.collinsdictionary.com/dictionary/english/blah
https://dictionary.cambridge.org/dictionary/english/blah-blah-blah
https://dictionary.cambridge.org/dictionary/english/blah
https://www.youtube.com/watch?v=mfJhMfOPWdE
https://www.thefreedictionary.com/blah
https://en.wiktionary.org/wiki/blah
https://www.lexico.com/definition/blah
https://www.yourdictionary.com/blah
https://en.wikipedia.org/wiki/Blah
https://www.definitions.net/definition/BLAH
https://www.thesaurus.com/browse/blah
https://www.macmillandictionary.com/dictionary/british/blah-blah-blah
https://www.macmillandictionary.com/dictionary/british/blah
https://blahrecords.com/
https://blahrecords.com/pages/about
https://www.daysoftheyear.com/days/blah-blah-blah-day/
https://www.facebook.com/blahrecords/
https://twitter.com/blahrecords?lang=en
https://uk.mrswordsmith.com/products/blah-b

KeyboardInterrupt: 

In [8]:
from googleapiclient.discovery import build   #Import the library
api_key = "my-secret-api-key"
cse_id = "my-secret-custom-search-id "
def google_query(query, api_key, cse_id, **kwargs):
    query_service = build("customsearch", 
                          "v1", 
                          developerKey=api_key
                          )  
    query_results = query_service.cse().list(q=query,    # Query
                                             cx=cse_id,  # CSE ID
                                             **kwargs    
                                             ).execute()
    return query_results['items']
my_results_list = []
my_results = google_query("apple iphone news 2019",
                          api_key, 
                          cse_id, 
                          num = 10
                          )
for result in my_results:
    my_results_list.append(result['link'])
    print(result['link'])

HttpError: <HttpError 400 when requesting https://www.googleapis.com/discovery/v1/apis/customsearch/v1/rest?key=my-secret-api-key returned "API key not valid. Please pass a valid API key.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'customsearch.googleapis.com'}}]">

In [3]:
lib_path = data_dir + word_rep_dir + "glove.42B.300d.txt"
lib = None

In [5]:
del lib
lib = pd.read_table(lib_path, header=None, sep=' ', encoding="utf8", index_col=0, quoting=csv.QUOTE_NONE)

In [73]:
lib.shape

(1917494, 300)

In [11]:
word = "intricate"
print(list(lib.index).index(word))
# list(lib.loc[word])

13777


In [12]:
word in lib.index

True

In [16]:
lib.index[13777]

'intricate'

In [21]:
lib.index.get_loc(word)

13777

In [46]:
all(lib.iloc[13777] == lib.loc[word])

True

In [72]:
# Test how long it takes (in milliseconds) to look up
# n_samples using the key index and the integer count index
n_samples = 300
times = []
indices = [np.random.randint(0, lib.shape[0]) for i in range(n_samples)]
for i in range(n_samples):
    j = indices[i]
    t1 = time.time()
    a = lib.iloc[j]
    t2 = time.time()
    times += [ t2 - t1 ]
print("Using iloc: " + str(np.mean(times) * 1000))
times = []
keys = [lib.index[np.random.randint(0, lib.shape[0])] for i in range(n_samples)]
for i in range(n_samples):
    j = keys[i]
    t1 = time.time()
    a = lib.loc[j]
    t2 = time.time()
    times += [ t2 - t1 ]
print("Using loc: " + str(np.mean(times) * 1000))

Using iloc: 0.0634121894836
Using loc: 30.3172707558


In [10]:
dataset = Dataset(combined_csv)
df = dataset.load()
np.arange(0, df.shape[0], 12)

array([ 0, 12, 24, 36, 48])